Importing libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
from keras.models import Sequential, Model
from keras.layers import Conv2D,Dense,Activation,MaxPooling2D,Flatten,BatchNormalization,Dropout,InputLayer
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

Function for creating CNN Model using parameters given

In [ ]:
def create_cnn_model(filters, fil_size, dense_neuron, batch_norm, dropout, act, padd, lr):
    keras.backend.clear_session()
    model = Sequential()
    model.add(InputLayer(input_shape=(227,227,3))) #Adding Input layer with image size as(227,227) with RGB channel

    #Adding 5 convolution Layers in a loop
    for i in range(5):
        model.add(Conv2D(filters[i],fil_size[i],padding=padd))
        if batch_norm:
            model.add(BatchNormalization())   
        model.add(Activation(act))
        model.add(MaxPooling2D(pool_size=(2,2)))
    
    #Adding the Dense layers as per the parameters passed
    model.add(Flatten())
    model.add(Dense(dense_neuron))
    model.add(Dropout(dropout))
    model.add(Activation("relu"))

    model.add(Dense(10,activation='softmax'))
    adam = tf.keras.optimizers.Adam(learning_rate=lr)
    model.compile(optimizer=adam,loss='categorical_crossentropy',metrics=['accuracy'])

    return model

Definition Function for preparing DATA:-

In [ ]:
def dataProcess(BATCH_SIZE, data_aug=True):
    if(data_aug):
        train_datagen = ImageDataGenerator(
            rescale=1./255,
            rotation_range=90,
            shear_range=0.2,
            zoom_range=0.2,
            validation_split=0.1,
            horizontal_flip=True)
    else:
        train_datagen = ImageDataGenerator( rescale=1./255, validation_split=0.1)

    test_datagen = ImageDataGenerator(rescale=1./255)

    train_generator = train_datagen.flow_from_directory(
        '/content/drive/MyDrive/DL_DATA_iNat/train',
        subset='training',
        target_size=(227,227),
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        shuffle = True,
        seed = 45)

    val_generator = train_datagen.flow_from_directory(
        '/content/drive/MyDrive/DL_DATA_iNat/train',
        subset = 'validation',
        target_size=(227,227),
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        shuffle = True,
        seed = 45)

    test_generator = test_datagen.flow_from_directory(
        '/content/drive/MyDrive/DL_DATA_iNat/val',
        target_size=(227,227),
        batch_size=BATCH_SIZE,
        class_mode='categorical')
    
    return train_generator, val_generator, test_generator
    

In [ ]:
#--------Fetching dataset---------------------------------
BATCH_SIZE = 32
train_gen, val_gen, test_gen = dataProcess(BATCH_SIZE, True)

Found 9000 images belonging to 10 classes.
Found 999 images belonging to 10 classes.
Found 2000 images belonging to 10 classes.


# **Train & Save Best Model configuration**

In [ ]:
fil=[32,64,128,256,256]
fils=[(3,3),(3,3),(3,3),(3,3),(3,3)]
d=0.20
batch_norm=True
epochs=20
d_n=1024
ler=0.0001

best = create_cnn_model(fil,fils,d_n,batch_norm,d,'relu',"same",ler)

In [ ]:
best.fit( train_gen, 
         steps_per_epoch = train_gen.samples // BATCH_SIZE, 
         validation_data = val_gen, 
         validation_steps = val_gen.samples // BATCH_SIZE, 
         epochs = epochs)

Epoch 1/20
281/281 [==============================] - 294s 1s/step - loss: 2.2735 - accuracy: 0.2316 - val_loss: 2.4233 - val_accuracy: 0.1109
Epoch 2/20
281/281 [==============================] - 279s 991ms/step - loss: 2.0091 - accuracy: 0.2890 - val_loss: 2.0233 - val_accuracy: 0.2812
Epoch 3/20
281/281 [==============================] - 272s 967ms/step - loss: 1.9540 - accuracy: 0.3063 - val_loss: 1.9614 - val_accuracy: 0.3014
Epoch 4/20
281/281 [==============================] - 269s 957ms/step - loss: 1.9191 - accuracy: 0.3217 - val_loss: 1.9124 - val_accuracy: 0.3246
Epoch 5/20
281/281 [==============================] - 269s 958ms/step - loss: 1.8874 - accuracy: 0.3305 - val_loss: 1.9090 - val_accuracy: 0.3357
Epoch 6/20
281/281 [==============================] - 264s 938ms/step - loss: 1.8569 - accuracy: 0.3419 - val_loss: 1.8922 - val_accuracy: 0.3317
Epoch 7/20
281/281 [==============================] - 262s 933ms/step - loss: 1.8287 - accuracy: 0.3512 - val_loss: 1.8466 - va

In [ ]:
best.save('model-best-41.h5')

In [ ]:
best.evaluate(test_gen,steps= test_gen.n/test_gen.batch_size)

62/62 [==============================] - 458s 7s/step - loss: 1.7713 - accuracy: 0.4085


[1.7713334560394287, 0.40849998593330383]